<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6ea7946a349fb753a35b4480b009de84e46acb54ef0dae59cbce252277283680
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-17 09:43:02
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 11.17 K (0.08%)
Current PnL: -20.97 L (-13.97%)
CY Booked + Current PnL: -9.07 L (-6.04%)
-------------------
Total profit:  1.85 L
Total loss:  -22.81 L
-------------------
Total Booked + Current PnL: 17.91 L (14.55%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.47%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.13 L (62.72%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,57.0,X-LC,3.87,214254.0,9754.0,13541.0,-0.35,4.77,6.32,11.39,10.0,0.72,1.54,14.15,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,97.65,52.0,M-SC,1.63,85143.0,-15634.0,15726.0,-0.63,-15.51,18.47,0.09,245.0,-0.99,0.61,11.86,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.58,59.0,X-MC,1.32,213000.0,21258.0,16508.0,0.31,11.09,7.75,19.69,99.0,1.29,1.53,18.64,XY25,NTT,AC
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.32,148052.0,11919.0,19202.0,-0.47,8.76,12.97,22.86,91.0,0.62,1.06,22.47,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-12.16,60.0,X-LC,2.83,219539.0,9173.0,19605.0,0.64,4.36,8.93,13.68,37.0,0.47,1.57,21.60,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-12.80,64.0,X-LC,5.29,220176.0,-31592.0,84812.0,3.83,-12.55,38.52,21.13,27.0,-0.37,1.58,18.43,X40,ATH,PAINTS
84,WHIRLPOOL,2270.00,-43.57,57.0,M-SC,4.43,95392.0,3894.0,74854.0,2.75,4.26,78.47,86.07,223.0,0.05,0.68,38.53,XR,NTT,DURABLES
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,29.16,81600.0,-31949.0,72004.0,1.80,-28.14,88.24,35.27,268.0,-0.44,0.59,99.72,XR,NTT,HOTELS
44,KANSAINER,340.00,-67.42,56.0,H-SC,1.82,224487.0,-45180.0,81511.0,1.74,-16.75,36.31,13.47,138.0,-0.55,1.61,14.15,XY24,NTT,PAINTS
67,STARHEALTH,761.00,21.75,71.0,H-SC,9.17,271184.0,-9939.0,142805.0,1.64,-3.54,52.66,47.26,144.0,-0.07,1.94,45.29,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-14.25,45.0,M-LC,5.67,150753.0,-192.0,109643.0,-4.20,-0.13,72.73,72.51,53.0,-0.00,1.08,5.89,XR,NTT,IT
7,ATULAUTO,844.00,3645.95,50.0,M-SC,3.76,168215.0,-28917.0,118743.0,-1.69,-14.67,70.59,45.57,236.0,-0.24,1.21,19.26,XY24,NTT,AUTO
39,INFY,2275.00,-20.67,41.0,X-LC,7.47,308637.0,-4251.0,175923.0,-1.53,-1.36,57.00,54.87,3.0,-0.02,2.21,5.20,X40,BTT,IT
14,BSOFT,836.99,-22.54,33.0,H-SC,15.10,86955.0,-53698.0,126476.0,-1.06,-38.18,145.45,51.74,131.0,-0.42,0.62,0.00,XR,ATH,IT
63,SHALBY,327.00,1175.92,54.0,M-SC,23.37,173423.0,-7720.0,53189.0,-0.99,-4.26,30.67,25.10,235.0,-0.15,1.24,38.98,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,18.69,149895.0,675.0,143509.0,-0.2,0.45,95.74,96.62,208.0,0.00,1.08,62.22,XY25,NTT,FINANCE
59,SAIL,228.0,43.54,49.0,M-MC,12.14,226012.0,1050.0,166593.0,-0.1,0.47,73.71,74.53,192.0,0.01,1.62,32.87,XY24,BTT,STEEL
85,WIPRO,420.0,-14.25,45.0,M-LC,5.67,150753.0,-192.0,109643.0,-4.2,-0.13,72.73,72.51,53.0,-0.00,1.08,5.89,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,12.32,131436.0,-11043.0,30559.0,-0.63,-7.75,23.25,13.70,163.0,-0.36,0.94,55.13,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,97.65,52.0,M-SC,1.63,85143.0,-15634.0,15726.0,-0.63,-15.51,18.47,0.09,245.0,-0.99,0.61,11.86,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,56.0,H-SC,1.82,224487.0,-45180.0,81511.0,1.74,-16.75,36.31,13.47,138.0,-0.55,1.61,14.15,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,44.0,H-SC,2.32,140012.0,-35891.0,77917.0,0.80,-20.40,55.65,23.89,149.0,-0.46,1.00,20.92,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.38,45.0,H-MC,7.33,102382.0,-29453.0,71678.0,-0.28,-22.34,70.01,32.03,98.0,-0.41,0.73,14.68,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,48.0,M-LC,6.23,205590.0,2420.0,39720.0,-0.98,1.19,19.32,20.74,55.0,0.06,1.47,6.80,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,49.0,M-MC,12.14,226012.0,1050.0,166593.0,-0.10,0.47,73.71,74.53,192.0,0.01,1.62,32.87,XY24,BTT,STEEL
31,HINDZINC,730.22,36.19,66.0,M-LC,10.08,221589.0,16513.0,96059.0,0.53,8.05,43.35,54.89,52.0,0.17,1.59,34.09,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,53.0,M-LC,8.79,156986.0,16146.0,28352.0,-0.53,11.46,18.06,31.59,66.0,0.57,1.13,40.99,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.25,192404.0,12596.0,99819.0,-0.26,7.01,51.88,62.52,88.0,0.13,1.38,43.48,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,66.0,M-LC,10.08,221589.0,16513.0,96059.0,0.53,8.05,43.35,54.89,52.0,0.17,1.59,34.09,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.25,192404.0,12596.0,99819.0,-0.26,7.01,51.88,62.52,88.0,0.13,1.38,43.48,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-43.57,57.0,M-SC,4.43,95392.0,3894.0,74854.0,2.75,4.26,78.47,86.07,223.0,0.05,0.68,38.53,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.39,40.0,H-SC,7.91,124619.0,1283.0,130339.0,-0.21,1.04,104.59,106.72,119.0,0.01,0.89,24.66,AR,ATH,MISC
85,WIPRO,420.00,-14.25,45.0,M-LC,5.67,150753.0,-192.0,109643.0,-4.20,-0.13,72.73,72.51,53.0,-0.00,1.08,5.89,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,7.41,146089.0,-130434.0,240506.0,1.42,-47.17,164.63,39.81,54.0,-0.54,1.05,2.97,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,34.0,X-SC,38.66,51826.0,-13180.0,164107.0,-0.71,-20.27,316.65,232.18,198.0,-0.08,0.37,1.07,XY24,NTT,MISC
34,ICICIPRULI,790.00,-23.40,38.0,X-MC,2.22,131871.0,-3917.0,46669.0,-0.60,-2.88,35.39,31.48,107.0,-0.08,0.95,8.88,X40,ATH,INSURANCE
28,HAVELLS,2069.16,-7.03,39.0,X-MC,2.14,230350.0,-17381.0,92439.0,0.05,-7.02,40.13,30.30,92.0,-0.19,1.65,5.34,X40,ATH,ELECTRICAL
39,INFY,2275.00,-20.67,41.0,X-LC,7.47,308637.0,-4251.0,175923.0,-1.53,-1.36,57.00,54.87,3.0,-0.02,2.21,5.20,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,45.0,H-LC,0.40,155925.0,-30170.0,77651.0,-0.02,-16.21,49.80,25.51,15.0,-0.39,1.12,14.90,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.58,59.0,X-MC,1.32,213000.0,21258.0,16508.0,0.31,11.09,7.75,19.69,99.0,1.29,1.53,18.64,XY25,NTT,AC
28,HAVELLS,2069.16,-7.03,39.0,X-MC,2.14,230350.0,-17381.0,92439.0,0.05,-7.02,40.13,30.30,92.0,-0.19,1.65,5.34,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.32,148052.0,11919.0,19202.0,-0.47,8.76,12.97,22.86,91.0,0.62,1.06,22.47,X40,ATH,INSURANCE
41,ITC,452.00,-38.34,54.0,X-LC,2.69,199194.0,-944.0,21832.0,0.54,-0.47,10.96,10.44,4.0,-0.04,1.43,5.07,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,34.0,X-SC,38.66,51826.0,-13180.0,164107.0,-0.71,-20.27,316.65,232.18,198.0,-0.08,0.37,1.07,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,7.41,146089.0,-130434.0,240506.0,1.42,-47.17,164.63,39.81,54.0,-0.54,1.05,2.97,XY24,NTT,AUTO
73,TCS,4389.96,-29.97,45.0,X-LC,12.63,279133.0,-66823.0,133509.0,-0.04,-19.32,47.83,19.28,1.0,-0.50,2.00,3.19,X40,ATH,IT
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,6.04,179334.0,-21172.0,118325.0,-0.63,-10.56,65.98,48.45,90.0,-0.18,1.29,3.41,X40N,ATH,FINANCE
2,ACC,3906.00,-37.20,53.0,X-MC,3.29,186350.0,-51601.0,204258.0,0.18,-21.69,109.61,64.15,174.0,-0.25,1.34,4.53,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,34.0,X-SC,38.66,51826.0,-13180.0,164107.0,-0.71,-20.27,316.65,232.18,198.0,-0.08,0.37,1.07,XY24,NTT,MISC
55,RELAXO,1176.00,-39.78,42.0,X-SC,4.41,82137.0,-63023.0,138951.0,-0.97,-43.42,169.17,52.31,136.0,-0.45,0.59,9.95,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.29,41.0,X-SC,4.31,95151.0,-33519.0,78814.0,-0.11,-26.05,82.83,35.20,219.0,-0.43,0.68,9.21,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,49.0,X-SC,0.83,107548.0,5544.0,38384.0,0.27,5.44,35.69,43.06,122.0,0.14,0.77,23.48,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,48.0,X-SC,2.51,108885.0,-19053.0,66191.0,0.57,-14.89,60.79,36.84,143.0,-0.29,0.78,11.47,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-29.97,45.0,X-LC,12.63,279133.0,-66823.0,133509.0,-0.04,-19.32,47.83,19.28,1.0,-0.50,2.00,3.19,X40,ATH,IT
39,INFY,2275.00,-20.67,41.0,X-LC,7.47,308637.0,-4251.0,175923.0,-1.53,-1.36,57.00,54.87,3.0,-0.02,2.21,5.20,X40,BTT,IT
41,ITC,452.00,-38.34,54.0,X-LC,2.69,199194.0,-944.0,21832.0,0.54,-0.47,10.96,10.44,4.0,-0.04,1.43,5.07,X40,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.52,294724.0,-21118.0,133775.0,0.14,-6.69,45.39,35.67,5.0,-0.16,2.11,6.24,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,49.0,H-LC,7.08,249168.0,-12451.0,131660.0,-0.40,-4.76,52.84,45.57,7.0,-0.09,1.79,6.02,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6683.33,54.0,L-SC,6.23,74429.0,-19381.0,95999.0,-0.96,-20.66,128.98,81.67,269.0,-0.20,0.53,44.13,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,29.16,81600.0,-31949.0,72004.0,1.80,-28.14,88.24,35.27,268.0,-0.44,0.59,99.72,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,45.0,H-SC,9.50,90960.0,-7020.0,28625.0,0.12,-7.16,31.47,22.05,152.0,-0.25,0.65,32.22,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,12.32,131436.0,-11043.0,30559.0,-0.63,-7.75,23.25,13.70,163.0,-0.36,0.94,55.13,OX40N,NTT,BANKS
68,SURYODAY,240.00,55.70,52.0,H-SC,10.52,143647.0,-35424.0,101156.0,-0.17,-19.78,70.42,36.71,135.0,-0.35,1.03,42.53,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.69,149895.0,675.0,143509.0,-0.20,0.45,95.74,96.62,208.0,0.00,1.08,62.22,XY25,NTT,FINANCE
4,ANGELONE,3033.00,16.33,70.0,X-SC,8.81,205918.0,14912.0,42790.0,1.15,7.81,20.78,30.21,157.0,0.35,1.48,30.32,X40N,NTT,FINANCE
31,HINDZINC,730.22,36.19,66.0,M-LC,10.08,221589.0,16513.0,96059.0,0.53,8.05,43.35,54.89,52.0,0.17,1.59,34.09,X5K,ATH,METALS
67,STARHEALTH,761.00,21.75,71.0,H-SC,9.17,271184.0,-9939.0,142805.0,1.64,-3.54,52.66,47.26,144.0,-0.07,1.94,45.29,XY24,NTT,INSURANCE
63,SHALBY,327.00,1175.92,54.0,M-SC,23.37,173423.0,-7720.0,53189.0,-0.99,-4.26,30.67,25.10,235.0,-0.15,1.24,38.98,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.17
2,50,75.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.34
LC    33.81
MC    21.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.58
X40      14.76
XY25     12.35
XR       10.89
X40N     10.23
AR        8.16
OX40N     7.62
X200      1.83
X5K       1.59
SR        1.12
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.47
X-LC    22.77
X-MC    16.12
M-SC    12.74
M-LC     5.27
H-LC     4.74
X-SC     4.67
H-MC     3.78
M-MC     1.62
L-SC     1.46
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.29,-3.11,37.09
IT,12.81,-17.51,80.93
FINANCE,12.20,-11.23,61.65
MISC,6.85,-20.13,86.69
BANKS,6.32,-13.20,72.75
PAINTS,5.68,-16.61,34.00
ELECTRICAL,5.34,-10.62,49.43
INSURANCE,3.95,-0.35,37.86
AUTO,2.85,-44.96,114.26


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3310632.0,22
XR,1391219.0,14
AR,1191268.0,9
X40,776874.0,10
X40N,602128.0,8
OX40N,580626.0,10
XY25,462073.0,8
SR,264690.0,2
X5K,96059.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3444375.0,25
M-SC,1264133.0,15
X-MC,1082382.0,11
X-LC,1062360.0,13
X-SC,529237.0,6
H-MC,351764.0,3
H-LC,279567.0,3
M-LC,273774.0,4
L-SC,257363.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1189439.0      6
           XR         829114.0      7
           AR         821010.0      5
M-SC       XY24       688916.0      6
X-MC       XY24       585591.0      4
X-LC       X40        483277.0      5
           XY24       307367.0      2
X-SC       X40N       286316.0      4
H-SC       OX40N      272956.0      4
           SR         264690.0      2
X-MC       X40        214783.0      4
H-LC       AR         209311.0      2
X-LC       X40N       197487.0      3
H-MC       XY24       180267.0      1
L-SC       XR         168003.0      2
M-MC       XY24       166593.0      1
X-SC       XY24       164107.0      1
X-MC       XY25       163683.0      2
M-SC       AR         160947.0      2
           OX40N      146632.0      4
           XY25       143509.0      1
           XR         124129.0      2
X-MC       X40N       118325.0      1
M-LC       XR         109643.0      1
H-MC       XR          99819.0      1
M-LC       X5K         96059.0      1
L-SC       OX40N       89360.0      1
X-SC       X40         78814.0      1
X-LC       XY25        74229.0      3
H-MC       OX40N       71678.0      1
H-LC       X200        70256.0      1
H-SC       MH          67166.0      1
L-MC       XR          60511.0      1
L-LC       XY25        40932.0      1
M-LC       XY25        39720.0      1
           XY24        28352.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
